In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine-Review").getOrCreate()

# **Load Amazon Data into Spark DataFrame**

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
from pyspark.sql.functions import to_date
clean_df = df.dropna()

In [5]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

# **Vine Review Analysis**

In [6]:
# create new dataframe to retrieve all rows where total votes is => 20
high_total_df = vine_df.filter('total_votes>= 20')
high_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [7]:
# create a new dataframe to retrieve all rows where the number of helpful votes/total votes is >= 50%
most_helpful_df = high_total_df.filter('(helpful_votes/total_votes)>=0.5')
most_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [8]:
# create new dataframe that retrieves all the rows where review was part of the Vine program
paid_reviews_df = most_helpful_df.filter("vine == 'Y'")
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|         36|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|     

In [9]:
# create dataframe that retrieves all rows where review was not part of the vine program
unpaid_reviews_df = most_helpful_df.filter("vine == 'N'")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [10]:
# count of total number of reviews
total_reviews = most_helpful_df.count()
total_reviews


37993

In [11]:
# count of total number of 5 star reviews
total_5_reviews = most_helpful_df.filter("star_rating == 5").count()
total_5_reviews

20670

In [12]:
# count of  total number of 5 star reviews of paid vine reviews
total_5_paid_reviews = paid_reviews_df.filter("star_rating == 5").count()
total_5_paid_reviews

65

In [13]:
# count of total number of 5 star reviews of unpaid vine reviews
total_5_unpaid_reviews = unpaid_reviews_df.filter("star_rating == 5").count()
total_5_unpaid_reviews

20605

In [14]:
# calculate percentage of total 5 star reviews vs total reviews
total_5_percentage = total_5_reviews/total_reviews * 100
total_5_percentage

54.40475877135261

In [15]:
# calculate percentage of 5 star reviews for paid vine reviews vs total 5 star reviews
percentage_5_paid_reviews = total_5_paid_reviews/total_5_reviews * 100
percentage_5_paid_reviews

0.3144654088050315

In [16]:
# calculate percentage of 5 star reviews for unpaid vine reviews vs total 5 star reviews
percentage_5_unpaid_reviews = total_5_unpaid_reviews/total_5_reviews * 100
percentage_5_unpaid_reviews

99.68553459119497

In [17]:
# calculate percentage of total paid 5 star reviews to total reviews
percentage_5_paid_to_total = total_5_paid_reviews/total_reviews * 100
percentage_5_paid_to_total

0.17108414707972522

In [18]:
# calculate percentage of total unpaid 5 star reviews to total reviews
percentage_5_unpaid_to_total = total_5_unpaid_reviews/total_reviews * 100
percentage_5_unpaid_to_total

54.233674624272886

In [19]:
# count of total paid reviews
total_paid_reviews = paid_reviews_df.count()
total_paid_reviews

170

In [20]:
# count of total unpaid reviews
total_unpaid_reviews = unpaid_reviews_df.count()
total_unpaid_reviews

37823

In [21]:
# percentage of paid 5 star reviews to total paid reviews
percent_paid_reviews_with_5 = total_5_paid_reviews/total_paid_reviews *100
percent_paid_reviews_with_5

38.23529411764706

In [22]:
# percentage of unpaid 5 star reviews to total unpaid reviews
percent_unpaid_reviews_with_5 = total_5_unpaid_reviews/total_unpaid_reviews *100
percent_unpaid_reviews_with_5

54.477434365333266